### Environment and Compute targets

The runtime context for each experiment run consists of two elements:
- The _environment_
- The _compute target_

#### Environments

Python runs are based on _virtual environments_ in which both the python version and packages are defined. In most, Python installation, packages are installed and managed by Conda or Pip.

To improve portability, we usually create environment in docker containers hosted in compute targets.

In general, AzureML handles the creation of this environment through the docker container and manages packages and python. It encapsulates the environment in the _Environment_ class which can be registered in the workspace.

Creating an environment from a specification file

In [1]:
from azureml.core import Environment

env = Environment(name = 'training_environment',
                  file_path = 'environment.yml')

Creating an environment from an existing Conda environment

In [12]:
env = Environment.from_existing_conda_environment(name='azure_env',
                                                  conda_environment_name='azure_env')

FileNotFoundError: [WinError 2] The system cannot find the file specified

Creating an environment by specifying packages

In [3]:
from azureml.core.conda_dependencies import CondaDependencies

env = Environment('training_environment')
deps = CondaDependencies.create(conda_packages = ['scikit-learn','pandas','numpy'],
                                pip_packages=['azureml-defaults'])
env.python.conda_dependencies = deps

Experiments usually use docker containers to create envs, otherwise the environment would be created in computer target (use_docker = False).

In [14]:
from azureml.core import Experiment, ScriptRunConfig
from azureml.core.runconfig import DockerConfiguration

docker_config = DockerConfiguration(use_docker=True)

script_config = ScriptRunConfig(source_directory='Script',
                                script='1_Script.py',
                                environment=env,
                                docker_runtime_config=docker_config)

AzureML has multiple docker's images which are chosen based on the type of compute target: indeed, if you are using a compute target with GPU, a docker's image with CUDA will be chosen.
Moreover, it is possible to create and register your own docker image.

In [ ]:
# env.docker.base_image='my-base-image'
# env.docker.base_image_registry='myregistry.azurecr.io/myimage'

On the other hand, you can add some additional layers based on the default image.

In [16]:
# env.docker.base_image=None
# env.docker.base_image_registry='myregistry.azurecr.io/myimage'

If your image includes the dependencies, you can flag it.

In [17]:
# env.python.user_managed_dependencies=True
# env.python.interpreter_path = '/opt/miniconda/bin/python'

Environment can be registered.

In [19]:
from azureml.core import Workspace

ws = Workspace.from_config()

env.register(workspace=ws)

{
    "assetId": "azureml://locations/francecentral/workspaces/b0c040f4-cad9-4180-843c-f18afbed9fe7/environments/training_environment/versions/1",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20221101.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "training_e

In [20]:
for env_name in Environment.list(workspace = ws):
    print('Name:', env_name)

Name: training_environment
Name: AzureML-VowpalWabbit-8.8.0
Name: AzureML-PyTorch-1.3-CPU
Name: AzureML-Dask-CPU
Name: AzureML-Dask-GPU
Name: AzureML-Triton
Name: AzureML-lightgbm-3.2-ubuntu18.04-py37-cpu
Name: AzureML-sklearn-0.24-ubuntu18.04-py37-cpu
Name: AzureML-sklearn-1.0-ubuntu20.04-py38-cpu
Name: AzureML-tensorflow-2.4-ubuntu18.04-py37-cuda11-gpu
Name: AzureML-tensorflow-2.6-ubuntu20.04-py38-cuda11-gpu
Name: AzureML-tensorflow-2.5-ubuntu20.04-py38-cuda11-gpu
Name: AzureML-tensorflow-2.7-ubuntu20.04-py38-cuda11-gpu
Name: AzureML-pytorch-1.7-ubuntu18.04-py37-cuda11-gpu
Name: AzureML-pytorch-1.8-ubuntu18.04-py37-cuda11-gpu
Name: AzureML-pytorch-1.9-ubuntu18.04-py37-cuda11-gpu
Name: AzureML-pytorch-1.10-ubuntu18.04-py38-cuda11-gpu
Name: AzureML-minimal-ubuntu18.04-py37-cpu-inference
Name: AzureML-responsibleai-0.20-ubuntu20.04-py38-cpu
Name: AzureML-responsibleai-0.21-ubuntu20.04-py38-cpu
Name: AzureML-PTA-pytorch-1.11-py38-cuda11.3-gpu
Name: AzureML-PTA-pytorch-1.11-py38-cuda11.5-

Retrieve a registered environment.

If you are trying to retrieve an environment which doesn't exist, AzureML will create it for you.

In [21]:
training_env = Environment.get(workspace=ws, name='training_environment')